In [1]:
import dask.array as da
import napari
import btrack
from tqdm.auto import tqdm
from skimage.io import imshow,  imsave, imread
import matplotlib.pyplot as plt
import numpy as np
import skimage as ski
import itertools
import json
import os
from octopusheavy import DaskOctopusHeavyLoader
plt.rcParams["figure.figsize"] = (15,7)
import warnings
warnings.filterwarnings("ignore")

# Loading images 

This time with multiple different experiments

In [2]:
root_path = '/run/user/30046150/gvfs/smb-share:server=data.thecrick.org,share=lab-gutierrezm/home/shared/Lung on chip/Light microscopy'
expt_IDs = ['co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/images/',
            'co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-VWF-iCAM1-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_70_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_150_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-AQP5-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_262_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day7_static/_20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_130_/images/',
            'mono-culture/iAT2AT1/for analysis_20221125/DAPI-CAV1-proSPC-ZO1/Day14_static/_20x_21-12-028A_A23456_Multichannel Z-Stack_20220818_253_/images/'
           ]

In [3]:
for i, expt in enumerate(expt_IDs):
    images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
    if 'MASK1' in [channel.name for channel in images.channels]:
        images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
        print(i, images.channels)
        tracks_fn = os.path.join(root_path, expt.replace('images/', 'tracks.hdf5'))
        print('Tracks exist:', os.path.exists(tracks_fn))

0 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: True
1 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: False
2 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: False
3 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: False
4 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: False
5 [<Channels.CH1: 1>, <Channels.CH2: 2>, <Channels.CH3: 3>, <Channels.CH4: 4>, <Channels.MASK1: 99>]
Tracks exist: False


In [9]:
i = 0
user = 'gabriel'
expt = expt_IDs[i]
images = DaskOctopusHeavyLoader(os.path.join(root_path, expt), remove_background=False)
with btrack.dataio.HDF5FileHandler(f'test_tracks_{i}.h5', "r", obj_type="obj_type_1") as hdf:
        tracks = hdf.tracks
#         tracks = [track for track in tracks if len(track) > 7]
        data, new_properties, new_graph = btrack.utils.tracks_to_napari(tracks, ndim=2)
print('Loaded expt:', expt)

[INFO][2022/12/02 10:23:58 AM] Opening HDF file: test_tracks_0.h5...
[INFO][2022/12/02 10:23:58 AM] Loading tracks/obj_type_1
[INFO][2022/12/02 10:23:58 AM] Loading LBEP/obj_type_1
[INFO][2022/12/02 10:23:59 AM] Loading objects/obj_type_1 (59348, 5) (59348 filtered: None)
[INFO][2022/12/02 10:24:01 AM] Closing HDF file: test_tracks_0.h5


Loaded expt: co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/images/


In [6]:
import enum
@enum.unique
class Channels(enum.Enum): ### this is unique to the above acquisition (20x_21-12-031B_A12456_Multichannel Z-Stack_20220811_113.vsi - 005 CF405, CF488, CF561, CF638.tif)
    DAPI = 1
    SPC = 2
    PDPN = 3
    ZO1 = 4

In [7]:
images.channels

[<Channels.CH1: 1>,
 <Channels.CH2: 2>,
 <Channels.CH3: 3>,
 <Channels.CH4: 4>,
 <Channels.MASK1: 99>]

# GABRIEL

In [14]:
output_dir = f'/home/dayn/data/lung_on_chip/pos_neg_dist/{expt}{user}'
if not os.path.exists(output_dir):
    os.makedirs(output_dir,)

In [15]:
output_dir

'/home/dayn/data/lung_on_chip/pos_neg_dist/co-culture/iVECs+iAT2AT1/Folder_20220808/A2-A5/analysis_20221125/DAPI-SPC-PDPN-ZO1/_20220808_kolf-WT_co-culture_20x_A2-A5_Multichannel Z-Stack_20220808_60_/images/gabriel'

In [17]:
tracks[0]

,ID,t,x,y,z,parent,root,state,generation,dummy,eccentricity,area,axis_minor_length,axis_major_length,orientation
0,93,0,1128.883301,3.766667,0.0,93,93,5,0,False,"(1,) array","(1,) array","(1,) array","(1,) array","(1,) array"


In [16]:
v = napari.Viewer()

v.add_image(images['CH1'], name = Channels.DAPI.name, blending = 'additive', colormap='blue', visible = False)
v.add_image(images['CH2'], name = Channels.SPC.name, blending = 'additive', colormap='green')
v.add_image(images['CH3'], name = Channels.PDPN.name, blending = 'additive', colormap='red')
v.add_image(images['CH4'], name = Channels.ZO1.name, blending = 'additive', colormap='gray', visible = False)
v.add_labels(images['MASK1'], visible=True)
tracker_layer = v.add_tracks(data, tail_length=0, visible = False)

@v.bind_key('t')
def SPC_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "SPC_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "SPC_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('b')
def SPC_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "SPC_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "SPC_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")

@v.bind_key('p')
def NKX21_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "NKX21_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "NKX21_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('n')
def NKX21_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "NKX21_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "NKX21_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('q')
def PDPN_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-2'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "PDPN_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "PDPN_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('c')
def PDPN_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-2'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "PDPN_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "PDPN_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")

Assistant skips harvesting pyclesperanto as it's not installed.


# GABRIEL

In [59]:
output_dir = '/home/dayn/data/LoC/pos_neg_dist/gabriel'

In [60]:
import napari

v = napari.Viewer()

v.add_image(ch0, name = Channels.DAPI.name, blending = 'additive', colormap='blue', visible = False)
v.add_image(ch1, name = Channels.NKX21.name, blending = 'additive', colormap='green')
v.add_image(ch2, name = Channels.PDPN.name, blending = 'additive', colormap='red')
v.add_image(ch3, name = Channels.ZO1.name, blending = 'additive', colormap='gray', visible = False)
p_pos_points = v.add_points(np.empty((0,3)), ndim = 3,name = 'PDPN Positive', face_color='green', symbol='o')
p_neg_points = v.add_points(np.empty((0,3)), ndim = 3,name = 'PDPN Negative', face_color='red', symbol='o')
n_pos_points = v.add_points(np.empty((0,3)), ndim = 3,name = 'NKX21 Positive', face_color= 'green', symbol='x')
n_neg_points = v.add_points(np.empty((0,3)), ndim = 3,name = 'NKX21 Negative', face_color='red', symbol='x')

v.add_labels(mask_stack, visible=False)
tracker_layer = v.add_tracks(data, tail_length=0, visible = False)

@v.bind_key('p')
def NKX21_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "NKX21_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "NKX21_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('n')
def NKX21_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "NKX21_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "NKX21_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('q')
def PDPN_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-2'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "PDPN_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "PDPN_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('c')
def PDPN_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-2'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "PDPN_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "PDPN_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")

Looks like there was no identifiable track there


Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 561, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/key_bindings.py", line 486, in on_key_press
    self.press_key(

Looks like there was no identifiable track there


Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 561, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/key_bindings.py", line 486, in on_key_press
    self.press_key(

Looks like there was no identifiable track there


...

Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 561, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/key_bindings.py", line 486, in on_key_press
    self.press_key(

Looks like there was no identifiable track there


Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 561, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/key_bindings.py", line 486, in on_key_press
    self.press_key(

Looks like there was no identifiable track there


Traceback (most recent call last):
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 561, in keyPressEvent
    self._keyEvent(self._vispy_canvas.events.key_press, ev)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/app/backends/_qt.py", line 622, in _keyEvent
    func(native=ev, key=key, text=str(ev.text()), modifiers=mod)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 453, in __call__
    self._invoke_callback(cb, event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 471, in _invoke_callback
    _handle_exception(self.ignore_callback_errors,
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/vispy/util/event.py", line 469, in _invoke_callback
    cb(event)
  File "/home/dayn/miniconda3/envs/aero/lib/python3.9/site-packages/napari/utils/key_bindings.py", line 486, in on_key_press
    self.press_key(

# Kim

In [124]:
output_dir = '/home/dayn/data/LoC/pos_neg_dist/kim'

In [125]:
import napari

v = napari.Viewer()

v.add_image(ch0, name = Channels.DAPI.name, blending = 'additive', colormap='blue', visible = False)
v.add_image(ch1, name = Channels.NKX21.name, blending = 'additive', colormap='green')
v.add_image(ch2, name = Channels.PDPN.name, blending = 'additive', colormap='red')
v.add_image(ch3, name = Channels.ZO1.name, blending = 'additive', colormap='gray', visible = False)
p_pos_points = v.add_points(np.empty((0,3)), ndim = 3,name = 'PDPN Positive', face_color='green', symbol='o')
p_neg_points = v.add_points(np.empty((0,3)), ndim = 3,name = 'PDPN Negative', face_color='red', symbol='o')
n_pos_points = v.add_points(np.empty((0,3)), ndim = 3,name = 'NKX21 Positive', face_color= 'green', symbol='x')
n_neg_points = v.add_points(np.empty((0,3)), ndim = 3,name = 'NKX21 Negative', face_color='red', symbol='x')

v.add_labels(mask_stack, visible=False)
tracker_layer = v.add_tracks(data, tail_length=0, visible = False, name = 'Kim')

@v.bind_key('p')
def NKX21_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "NKX21_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "NKX21_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('n')
def NKX21_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-1'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "NKX21_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "NKX21_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('q')
def PDPN_extract_positive_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-2'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "PDPN_positive.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "PDPN_positive_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")
@v.bind_key('c')
def PDPN_extract_negative_pixel_value(event=None):
    ### extract cell ID under cursor
    cell_ID = tracker_layer._get_value(position=v.cursor.position)
    ### get time frame
    time = v.cursor.position[0]
    try:
        positive_track = [track for track in tracks if track.ID == cell_ID][0]
    except:
        print('Looks like there was no identifiable track there')
    ### get index of cell instance of selected frame
    cell_index = positive_track.t.index(time)
    ### extract the pixel intensity of that segment
    px_int =  positive_track['mean_intensity-2'][cell_index]
    ### write to output file
    with open(os.path.join(output_dir, "PDPN_negative.csv"),"a") as f:
        f.write(f"{px_int}\n")
    with open(os.path.join(output_dir, "PDPN_negative_details.csv"),"a") as f:
        f.write(f"{cell_ID, time, px_int}\n")

# fixing error
Accidental storage of NKX21 location as PDPN values, need to extract coordinates and use to find NKX21

In [62]:
import pandas as pd

In [115]:
df = pd.read_csv('/home/dayn/data/LoC/pos_neg_dist/gabriel0/PDPN_negative_details.csv', names = ['Cell ID', 'Time', 'px_int'])
df["Cell ID"] = df["Cell ID"].astype(str)
df["Cell ID"] = df["Cell ID"].str.replace("(",'')
df["Cell ID"] = df["Cell ID"].astype(int)
df["px_int"] = df["px_int"].astype(str)
df["px_int"] = df["px_int"].str.replace(")",'')

In [116]:
df

,Cell ID,Time,px_int
0,2569,13.0,704.0744081172492
1,2572,13.0,634.6532066508314
2,2727,13.0,603.3637454981993
3,5772,13.0,390.5770787916537
4,5026,13.0,378.27298850574715
...,...,...,...
107,733,12.0,548.0607476635514
108,1720,12.0,783.5562719812426
109,2909,12.0,834.6669666966696
110,3868,12.0,417.65410385259634


In [117]:
df_fixed = df

In [118]:
new_px_int = []
for row in df.iterrows():
    cell_ID = row[1]['Cell ID']
    time = row[1]['Time']
    cell = [track for track in tracks if track.ID == cell_ID][0]
    cell_index = cell.t.index(time)
    knx21 = cell['mean_intensity-1'][cell_index]
    new_px_int.append(knx21)

In [119]:
df_fixed['knx21'] = new_px_int

In [120]:
df_fixed

,Cell ID,Time,px_int,knx21
0,2569,13.0,704.0744081172492,302.520857
1,2572,13.0,634.6532066508314,246.046318
2,2727,13.0,603.3637454981993,297.551020
3,5772,13.0,390.5770787916537,210.526627
4,5026,13.0,378.27298850574715,184.321839
...,...,...,...,...
107,733,12.0,548.0607476635514,219.192368
108,1720,12.0,783.5562719812426,202.330012
109,2909,12.0,834.6669666966696,318.413141
110,3868,12.0,417.65410385259634,175.871022


In [121]:
del df_fixed['px_int']

In [122]:
df_fixed

,Cell ID,Time,knx21
0,2569,13.0,302.520857
1,2572,13.0,246.046318
2,2727,13.0,297.551020
3,5772,13.0,210.526627
4,5026,13.0,184.321839
...,...,...,...
107,733,12.0,219.192368
108,1720,12.0,202.330012
109,2909,12.0,318.413141
110,3868,12.0,175.871022


In [123]:
df_fixed.to_csv('/home/dayn/data/LoC/pos_neg_dist/gabriel0/NKX21_negative_details.csv', header = False, index = False)
df_fixed['knx21'].to_csv('/home/dayn/data/LoC/pos_neg_dist/gabriel0/NKX21_negative.csv', header = False, index = False)

In [104]:
df = pd.read_csv('/home/dayn/data/LoC/pos_neg_dist/gabriel0/PDPN_positive_details.csv', names = ['Cell ID', 'Time', 'px_int'])
df["Cell ID"] = df["Cell ID"].astype(str)
df["Cell ID"] = df["Cell ID"].str.replace("(",'')
df["Cell ID"] = df["Cell ID"].astype(int)
df["px_int"] = df["px_int"].astype(str)
df["px_int"] = df["px_int"].str.replace(")",'')

In [105]:
df

,Cell ID,Time,px_int
0,3052,10.0,587.3253012048193
1,3035,10.0,784.6757400476353
2,630,10.0,643.4101540216772
3,1533,10.0,784.31015625
4,2642,10.0,490.74047718710756
5,874,10.0,514.3316816351812
6,108,10.0,640.6706161137441
7,889,10.0,625.2869080779944
8,2506,10.0,788.7857142857143
9,1538,10.0,751.7081174438688


In [106]:
df_fixed = df

In [107]:
new_px_int = []
for row in df.iterrows():
    cell_ID = row[1]['Cell ID']
    time = row[1]['Time']
    cell = [track for track in tracks if track.ID == cell_ID][0]
    cell_index = cell.t.index(time)
    knx21 = cell['mean_intensity-1'][cell_index]
    new_px_int.append(knx21)

In [108]:
df_fixed['knx21'] = new_px_int

In [109]:
df_fixed

,Cell ID,Time,px_int,knx21
0,3052,10.0,587.3253012048193,575.520482
1,3035,10.0,784.6757400476353,612.406261
2,630,10.0,643.4101540216772,566.470622
3,1533,10.0,784.31015625,696.246094
4,2642,10.0,490.74047718710756,582.109251
5,874,10.0,514.3316816351812,480.732425
6,108,10.0,640.6706161137441,494.828594
7,889,10.0,625.2869080779944,701.229805
8,2506,10.0,788.7857142857143,521.916667
9,1538,10.0,751.7081174438688,620.856649


In [110]:
del df_fixed['px_int']

In [111]:
df_fixed

,Cell ID,Time,knx21
0,3052,10.0,575.520482
1,3035,10.0,612.406261
2,630,10.0,566.470622
3,1533,10.0,696.246094
4,2642,10.0,582.109251
5,874,10.0,480.732425
6,108,10.0,494.828594
7,889,10.0,701.229805
8,2506,10.0,521.916667
9,1538,10.0,620.856649


In [113]:
df_fixed.to_csv('/home/dayn/data/LoC/pos_neg_dist/gabriel0/NKX21_positive_details.csv', header = False, index = False)
df_fixed['knx21'].to_csv('/home/dayn/data/LoC/pos_neg_dist/gabriel0/NKX21_positive.csv', header = False, index = False)